In [1]:
# install if needed
!pip install --upgrade google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.5/253.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.32.0
    Uninstalling google-cloud-bigquery-3.32.0:
      Successfully uninstalled google-cloud-bigquery-3.32.0


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Authenticate your gmail account
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Connecting to Project in BigQuery
from google.cloud import bigquery
client = bigquery.Client(project='***')

In [7]:
## function to Run Queries

def query_run(query_string):
  # Set up a dry run job configuration
  job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)

  # Submit the dry run query
  query_job = client.query(query_string, job_config=job_config)

  # Print the number of Mb that would be processed
  total_bytes = query_job.total_bytes_processed
  total_mb = total_bytes / (1024 ** 2)
  if total_mb<500:
    print(f"This query will process {total_mb:.2f} MB.")
    query_job = client.query(query_string)
    # Convert the query results into a pandas DataFrame
    df = query_job.to_dataframe()
    return df
  else:
    print('Error! Query is taking more than 500 Mbs! Please check the query')

In [ ]:
# upload the one beat slack message from google drive
query1 = """
SELECT message_channel_id,parent_message_ts, ts, user_id, masked_text, thread_ts
FROM ***.slack.new_masked_messages_conv_ts
WHERE subtype is NULL or subtype = 'thread_broadcast'
"""

new_df=query_run(query1)
new_df.head(2)

This query will process 16.55 MB.


,message_channel_id,parent_message_ts,ts,user_id,masked_text,thread_ts
0,C06GRG33Z1Q,1723657926.395449,1723657926.395449,U079QLGUP51,Ok perfecto,NaN
1,C04B5V49ZDX,1710943626.719819,1710943626.719819,U02GQ1HESE6,"<@U03PT8K5KEX> Thank you, happy to join the te...",NaN


In [10]:
# convert timestamps
new_df['ts']=pd.to_datetime(new_df['ts'], unit='s')
new_df['parent_message_ts']=pd.to_datetime(new_df['parent_message_ts'], unit='s')

<ipython-input-10-f4ad554ac55b>:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  new_df['ts']=pd.to_datetime(new_df['ts'], unit='s')
<ipython-input-10-f4ad554ac55b>:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  new_df['parent_message_ts']=pd.to_datetime(new_df['parent_message_ts'], unit='s')


In [11]:
# calculate the minimum and maximum ts per parent message and calculate the time length per parent message.
new_df['min_ts']=new_df.groupby('parent_message_ts')['ts'].transform('min')
new_df['max_ts']=new_df.groupby('parent_message_ts')['ts'].transform('max')
new_df.parent_message_ts.nunique()


16347

# **Calculate the number of replies and total time to resolution per parent_message_ts.**

In [12]:
# Calculate the number of replies per parent_message and also the time length by subtracting last ts from first ts.
stat_parent_message = new_df.groupby('parent_message_ts').agg({'ts': ['count', lambda x: x.max() - x.min()]})
stat_parent_message.columns = ['num_replies', 'time_length']
stat_parent_message.sort_values(by='num_replies', ascending=False).head().reset_index()

,parent_message_ts,num_replies,time_length
0,2025-01-03 15:47:16.152709007,97,35 days 21:35:57.953599930
1,2024-07-02 06:04:23.951159000,88,22 days 06:11:20.547430039
2,2023-12-27 16:23:59.912969112,87,4 days 00:00:44.804499865
3,2024-07-31 13:22:26.018099070,86,168 days 08:27:22.592550039
4,2024-03-18 23:13:09.365098953,81,7 days 15:39:36.813549996


In [13]:
# concevert the time_length of stat_parent_message dataframe from days into hours.
stat_parent_message['time_length'] = stat_parent_message['time_length'].dt.total_seconds() / 3600
stat_parent_message.sort_values(by='num_replies', ascending=False).head().reset_index()

,parent_message_ts,num_replies,time_length
0,2025-01-03 15:47:16.152709007,97,861.599432
1,2024-07-02 06:04:23.951159000,88,534.189041
2,2023-12-27 16:23:59.912969112,87,96.012446
3,2024-07-31 13:22:26.018099070,86,4040.456276
4,2024-03-18 23:13:09.365098953,81,183.660226


In [ ]:
# Extract a new dataframe containing user id U02NPDLQ0UE in user_id column or tagged in masked_text columns.
j_df = new_df[(new_df['masked_text'].str.contains('U02NPDLQ0UE'))]
j_df.head(3)

,message_channel_id,parent_message_ts,ts,user_id,masked_text,thread_ts,min_ts,max_ts
187,C06DWQL6SLE,2024-07-30 22:40:02.495129108,2024-07-30 22:40:02.495129108,U02HGDV9ZJ7,"<@U02NPDLQ0UE> <@U03DPMYDD9R> please, help us ...",1.722379e+09,2024-07-30 22:40:02.495129108,2024-08-01 14:37:03.637129068
195,C05HUGUBHD2,2023-10-29 11:28:40.679239035,2023-10-29 11:28:40.679239035,U031E2LJJET,We came to the stage that we need to create an...,1.698579e+09,2023-10-29 11:28:40.679239035,2023-11-07 08:04:45.148739100
336,C0790H8G90E,2024-11-15 23:45:56.466768980,2024-11-16 05:00:44.429429054,U05C9AY5L0L,<@U02NPDLQ0UE> Thanks for checking.\n\n<@U06C4...,1.731714e+09,2024-11-15 23:45:56.466768980,2024-11-16 05:47:31.691458941


In [ ]:
# extract the full replies from the new_df that the parent message ts is in josh_df.
full_j_df = new_df[new_df['parent_message_ts'].isin(j_df['parent_message_ts'])]
full_j_df.shape

(9647, 8)

In [ ]:
print(j_df.shape)
print(full_j_df.shape)
print(j_df.parent_message_ts.nunique())
print(full_j_df.parent_message_ts.nunique())

(1902, 8)
(9647, 8)
1006
1006


In [ ]:
# calculate the number of replies and resolution time per parent message.
stat_j = full_j_df.groupby('parent_message_ts').agg({'ts': ['count', lambda x: x.max() - x.min()]})
stat_j.columns = ['num_replies_j', 'time_length_j']
stat_j.sort_values(by='num_replies_j', ascending=False).head().reset_index()

,parent_message_ts,num_replies_j,time_length_j
0,2023-12-27 16:23:59.912969112,87,4 days 00:00:44.804499865
1,2024-07-31 13:22:26.018099070,86,168 days 08:27:22.592550039
2,2024-03-18 23:13:09.365098953,81,7 days 15:39:36.813549996
3,2023-08-24 15:16:19.883219004,62,3 days 15:15:39.412060022
4,2024-03-14 16:04:48.300599098,57,55 days 01:02:18.567039967


In [ ]:
# left join stat_josh on stat_parent message based on parent_message_ts.
stat_j_main = stat_parent_message.merge(stat_j, how='right', left_on='parent_message_ts', right_on='parent_message_ts')
stat_j_main.reset_index().head()


,parent_message_ts,num_replies,time_length,num_replies_j,time_length_j
0,2022-03-07 12:47:34.115818977,3,646.884887,3,26 days 22:53:05.592459917
1,2022-03-14 11:24:51.672439098,3,9.475425,3,0 days 09:28:31.528399945
2,2022-06-22 04:02:11.382498980,13,32.044676,13,1 days 08:02:40.834660053
3,2022-06-26 08:52:35.992918968,6,5.081185,6,0 days 05:04:52.265320063
4,2022-06-28 16:33:01.290328979,3,2.937940,3,0 days 02:56:16.584460021


### **Manager J. is tagged in 1006 parent messages .**

In [ ]:
# extract standalone messages
j_not_replied = stat_j_main[stat_j_main['num_replies'] == 1].reset_index()
j_not_replied.head(3)

,parent_message_ts,num_replies,time_length,num_replies_j,time_length_j
0,2022-06-29 14:29:05.385329008,1,0.0,1,0 days
1,2022-07-05 11:40:22.065989017,1,0.0,1,0 days
2,2022-07-20 12:48:42.563429117,1,0.0,1,0 days


In [ ]:
# using the parent_message_ts of the standalone message filiter the j_df.
j_standalone = j_df[j_df['parent_message_ts'].isin(j_not_replied['parent_message_ts'])].reset_index()
j_standalone.shape

(102, 9)

### **Out of 1006 parent messages 102 messages are standalone i.e. not received a reply.**

In [ ]:
# save the j_standalone dataframe to csv.
j_nonreplied = j_standalone.to_csv('j_standalone.csv')

In [ ]:
# extract messages that received more than one reply
j_replied = stat_j_main[stat_j_main['num_replies'] > 1].reset_index()
j_replied.head(3)

,parent_message_ts,num_replies,time_length,num_replies_j,time_length_j
0,2022-03-07 12:47:34.115818977,3,646.884887,3,26 days 22:53:05.592459917
1,2022-03-14 11:24:51.672439098,3,9.475425,3,0 days 09:28:31.528399945
2,2022-06-22 04:02:11.382498980,13,32.044676,13,1 days 08:02:40.834660053


In [ ]:
# how many parent messages have been completed within 24 hours.
j_quick_resolved = j_replied[j_replied['time_length'] < 24]
j_quick_resolved.shape[0] / j_replied.shape[0]
# 50% of the message was completed with in 24 hours. There rest took more than 24 hours.

0.4646017699115044

In [ ]:
# calculate the average num_replies and time_length for j_quick_resolved messages
print(j_quick_resolved['num_replies'].mean())
print(j_quick_resolved['time_length'].mean())

6.280952380952381
8.216466751731591


In [ ]:
# Extract how many parent messages from j_replied were close after 24 hours and calculate means.
j_slow_resolved = j_replied[j_replied['time_length'] > 24]
print(j_slow_resolved['num_replies'].mean())
print(j_slow_resolved['time_length'].mean())

14.270661157024794
275.63947044302563


In [ ]:
# identify parent message with urgent-related words in the j_df.
import re
from collections import Counter

# Extract the 'masked_text' column
masked_texts = j_df["masked_text"].dropna().astype(str)

# Define urgency-related words
urgency_keywords = [
    "urgent", "asap", "immediately", "priority", "critical", "important",
    "emergency", "high-priority",'Alert','As soon as possible','Deadline',
    'Escalate','Right away','Top priority','Escalation','Rush','Emergency',
    'Fast response','Can\'t wait','High importance','Must be done','Act fast',
    'Pressing'
]

# Count occurrences of urgency words
word_counts = Counter()

for text in masked_texts:
    words = re.findall(r'\b\w+\b', text.lower())  # Tokenize words
    for word in words:
        if word in urgency_keywords:
            word_counts[word] += 1

# Convert to DataFrame for better display
urgency_df_j = pd.DataFrame(word_counts.items(), columns=["Urgency Word", "Count"]).sort_values(by="Count", ascending=False)
urgency_df_j

,Urgency Word,Count
3,asap,47
1,important,37
0,priority,21
5,critical,13
4,urgent,12
2,immediately,8
6,emergency,1


In [ ]:
# how many messages have an urgent tag in the masked_text field of the josh_df? case=False ensures that it is not case sensetive.
urgent_keywords = [ "urgent", "asap", "immediately", "priority", "critical", "important",
    "emergency", "high-priority",'Alert','As soon as possible','Deadline',
    'Escalate','Right away','Top priority','Escalation','Rush','Emergency',
    'Fast response','Can\'t wait','High importance','Must be done','Act fast',
    'Pressing']
# urgent_messages = df1[df1["masked_text"].str.contains(r"emergency", case=False, na=False)]
df_urgent_j = full_j_df[full_j_df['masked_text'].str.lower().str.contains('|'.join(urgent_keywords), na=False)]
df_urgent_j.head(3)

,message_channel_id,parent_message_ts,ts,user_id,masked_text,thread_ts,min_ts,max_ts
1864,C07B1K4MWUR,2024-08-21 11:41:15.001288891,2024-08-22 18:30:24.844758987,U02SM3N219N,Task <[URL HIDDEN] HIDDEN]|Create digger for S...,1.724240e+09,2024-08-21 11:41:15.001288891,2024-08-28 19:50:52.513029099
2133,C06GG3QNTTJ,2024-02-27 21:42:47.107279062,2024-02-27 21:42:47.107279062,U04CVQHC550,Hi <!channel>\nIs there a way I can get the s3...,1.709070e+09,2024-02-27 21:42:47.107279062,2024-03-05 06:16:15.672528982
2738,C07P81SMQRM,2025-02-05 12:50:44.873668909,2025-02-05 12:50:44.873668909,U07RCCRLYTH,"Team (<@U06C4BHN2FL> <@U04VDGAD067>), can we p...",1.738760e+09,2025-02-05 12:50:44.873668909,2025-02-06 06:48:39.288459063


In [ ]:
# extract the full df_urgent_josh from the full_j_df based on the parent message ts.
full_df_urgent_j = full_j_df[full_j_df['parent_message_ts'].isin(df_urgent_j['parent_message_ts'])]
full_df_urgent_j.head(2)

,message_channel_id,parent_message_ts,ts,user_id,masked_text,thread_ts,min_ts,max_ts
106,C06P285F1UM,2024-06-07 15:28:10.465949059,2024-06-10 09:04:39.324289083,U036S6AHBLG,haha that's what I said - it's a bug. You need...,1.717774e+09,2024-06-07 15:28:10.465949059,2024-06-30 16:09:18.020148993
156,C07P81SMQRM,2025-02-03 03:48:51.476819038,2025-02-03 15:43:49.035809040,U07RCCRLYTH,"Haha, thank you <@U02NPDLMK70>, I feel more li...",1.738555e+09,2025-02-03 03:48:51.476819038,2025-02-04 12:21:27.695229053


In [ ]:
# count the number of replies per parent_message_ts in df_urgent_j.
stat_urgent_j = df_urgent_j.groupby('parent_message_ts')['ts'].count()
stat_urgent_j.reset_index().head()

,parent_message_ts,ts
0,2022-06-28 16:33:01.290328979,1
1,2022-06-29 10:09:13.184149027,1
2,2022-06-30 00:56:51.026209116,1
3,2022-07-06 22:59:32.668278933,2
4,2022-07-18 15:11:32.612978935,1


In [ ]:
stat_urgent_j.shape

(241,)

### **Using the urgent_related words shown above 241 parent messages (~24%) out of 1006 were identified as urgent message in the J. dataframe. The question is weather these urgent tagged messages are really urgent?**

In [ ]:
# merge stat_urgent_j dataframe with stat_parent_message based on the parent_message_ts.
urgent_j_merged = stat_parent_message.merge(stat_urgent_j, how='right', left_on='parent_message_ts', right_on='parent_message_ts')
urgent_j_merged.sort_values(by='num_replies', ascending=False).reset_index().head()

,parent_message_ts,num_replies,time_length,ts
0,2023-12-27 16:23:59.912969112,87,96.012446,2
1,2024-07-31 13:22:26.018099070,86,4040.456276,3
2,2024-03-14 16:04:48.300599098,57,1321.038491,1
3,2024-02-01 22:04:56.377738953,56,623.619236,1
4,2024-04-02 23:15:30.321119070,55,542.337324,4


In [ ]:
# count the number of urgent_j_merged more than one replies.
urgent_j_merged[urgent_j_merged['num_replies'] > 1].shape

(234, 3)

In [ ]:
# How many of the urgent_j_merged messages are standalone.
urgent_standalone = urgent_j_merged[urgent_j_merged['num_replies'] == 1].reset_index()
urgent_standalone.shape

(7, 4)

In [ ]:
# using the parent_message_ts of urgent_standalone message extract the data from j_df.
urgent_standalone_df = j_df[j_df['parent_message_ts'].isin(urgent_standalone['parent_message_ts'])]
urgent_standalone_df.head(2)

,message_channel_id,parent_message_ts,ts,user_id,masked_text,thread_ts,min_ts,max_ts
11969,C03P08YAJE6,2024-09-13 12:03:16.516968966,2024-09-13 12:03:16.516968966,U02NPDLMK70,<@U02G46UTNJ3> since almost on daily basis thi...,NaN,2024-09-13 12:03:16.516968966,2024-09-13 12:03:16.516968966
19739,C057Z9AUJM9,2023-06-27 16:11:15.544688940,2023-06-27 16:11:15.544688940,U02NPDLMK70,<@U032ELWEN87> let's jump to a call with <@U02...,NaN,2023-06-27 16:11:15.544688940,2023-06-27 16:11:15.544688940


In [ ]:
# From urgent_j_merged extract message that have time length > 24 hours and <= 24 hours.
true_urgent_j = urgent_j_merged[urgent_j_merged['time_length'] <= 24].reset_index()
false_urgent_j = urgent_j_merged[urgent_j_merged['time_length'] > 24].reset_index()
print(true_urgent_j.shape)
print(false_urgent_j.shape)

(73, 4)
(168, 4)


In [ ]:
# count the number of true urgent message with num_replies greater than 1.
true_urgent_j[true_urgent_j['num_replies'] > 1].shape

(66, 4)

In [ ]:
# what is the proportion of true urgent messages from the entire urgent message with more than 1 reply.
true_urgent_j[true_urgent_j['num_replies'] > 1].shape[0] / urgent_j_merged[urgent_j_merged['num_replies'] > 1].shape[0]

0.28205128205128205

In [ ]:
# using the parent_message_ts of the true_urgent_josh dataframe extract messages from j_df.
true_urgent_j_df = full_j_df[full_j_df['parent_message_ts'].isin(true_urgent_j['parent_message_ts'])]
true_urgent_j_df.head(2)
# save as csv file
true_urgent = true_urgent_j_df.to_csv('true_urgentj.csv')
true_urgent_j_df.parent_message_ts.nunique()
true_urgent_j_df.shape

(596, 8)

In [ ]:
# using the parent_message_ts of the false_urgent_j dataframe extract messages from j_df.
false_urgent_j_df = full_j_df[full_j_df['parent_message_ts'].isin(false_urgent_j['parent_message_ts'])]
false_urgent_j_df.head(2)
# save as csv file
false_urgent = false_urgent_j_df.to_csv('false_urgentj.csv')
false_urgent_j_df.shape

(3295, 8)

### **182 of the 264 presumed urgent_massages took more than 24 hours for resolution. So, they can be regarded as a false urgent messages. Intrerstingly, the average replies of the two categories is also considerably different. 7.6 replies for true urgent and 19.3 replies for false urgent.**

In [ ]:
# calculate the average num_replies of true and false_urgent_j messages.
print(true_urgent_j['num_replies'].mean())
print(false_urgent_j['num_replies'].mean())

8.164383561643836
19.613095238095237


## **Approval analysis of J.**

In [ ]:
# Define a function to check if a message contains an approval statement
def contains_approval(text):
    approval_keywords = ["approved", "okay", "yes", "confirmed", "go ahead", "sounds good", "fine", "agreed"]
    return any(word in text.lower() for word in approval_keywords)

# Filter messages by user ID U02NPDLQ0UE i.e manager J.
true_urgent_user_df = true_urgent_j_df[true_urgent_j_df["user_id"] == "U02NPDLQ0UE"]
false_urgent_user_df = false_urgent_j_df[false_urgent_j_df["user_id"] == "U02NPDLQ0UE"]

# Count approvals in true and false urgent datasets
true_urgent_approvals = sum(true_urgent_user_df["masked_text"].apply(contains_approval))
false_urgent_approvals = sum(false_urgent_user_df["masked_text"].apply(contains_approval))

# Create a comparison dataframe
approval_comparison = pd.DataFrame({
    "Category": ["True Urgent", "False Urgent"],
    "Total Messages by User": [true_urgent_user_df.shape[0], false_urgent_user_df.shape[0]],
    "Approvals Given": [true_urgent_approvals, false_urgent_approvals],
    "Ratio": [true_urgent_approvals / true_urgent_user_df.shape[0], false_urgent_approvals / false_urgent_user_df.shape[0]]
})
approval_comparison

,Category,Total Messages by User,Approvals Given,Ratio
0,True Urgent,88,3,0.034091
1,False Urgent,280,33,0.117857


# **Funnel Analysis**

In [ ]:
total_messages = stat_parent_message.shape[0]
j_messages = stat_j.shape[0]
urgent_messages = stat_urgent_j.shape[0]
false_urgent_messages = false_urgent_j.shape[0]
# create a dataframe of the above four values to creat a funnel figure.
# Create a dictionary with the given variables and their values
summary_data = {
    "Variable": ["Total Messages", "J. Messages", "Urgent Messages", "False Urgent Messages"],
    "Value": [total_messages, j_messages , urgent_messages, false_urgent_messages]  # Placeholder values to be filled once data is available
}

# Convert to a DataFrame
summary_df = pd.DataFrame(summary_data)
summary_df

,Variable,Value
0,Total Messages,16347
1,Josh Messages,1006
2,Urgent Messages,241
3,False Urgent Messages,168


In [ ]:
import plotly.graph_objects as go

# Define funnel stages and corresponding values
stages = ["Total Messages", "J. Messages", "Urgent Messages", "False Urgent Messages"]
values = [total_messages, j_messages, urgent_messages, false_urgent_messages]  # Replace with actual values

# Create the funnel chart
fig = go.Figure(go.Funnel(
    y=stages,
    x=values,
    textinfo="value+percent initial"
))

# Customize layout
fig.update_layout(
    title="J. Urgency Funnel Analysis",
    xaxis_title="Message Count",
    yaxis_title="Stages"
)

# Show the figure
fig.show()


In [ ]:
# from josh_df extract the messages that are not urgent messages.
not_urgent_j = full_j_df[~full_j_df['parent_message_ts'].isin(df_urgent_j['parent_message_ts'])]
not_urgent_j.head(2)
not_urgent_j.shape

(5756, 8)

In [ ]:
# count the number of replies per parent message non urgent messages
stat_not_urgent_j = not_urgent_j.groupby('parent_message_ts')['ts'].count()
stat_not_urgent_j.reset_index().sort_values(by='ts', ascending=False).head()

,parent_message_ts,ts
482,2024-03-18 23:13:09.365098953,81
207,2023-08-24 15:16:19.883219004,62
351,2024-01-04 18:20:22.654078960,52
328,2023-12-10 11:04:44.715538979,49
131,2023-06-18 07:08:50.820439100,46
